## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-12-11-52-00 +0000,wsj,Armani’s Will Seeks Sale of Fashion Empire or ...,https://www.wsj.com/business/armanis-will-seek...
1,2025-09-12-11-50-40 +0000,bbc,Prince Harry makes surprise Ukraine visit to s...,https://www.bbc.com/news/articles/c3dr859vxxjo...
2,2025-09-12-11-47-24 +0000,bbc,UK economy saw zero growth in July,https://www.bbc.com/news/articles/c203edl1zq3o...
3,2025-09-12-11-45-00 +0000,wsj,"Confusion, Anger, Relief: Korean Engineer Tell...",https://www.wsj.com/world/asia/confusion-anger...
4,2025-09-12-11-32-34 +0000,nypost,Charlie Kirk assassin likely lacks social skil...,https://nypost.com/2025/09/12/us-news/charlie-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
32,kirk,64
31,charlie,54
53,trump,37
63,shooting,17
172,assassination,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
54,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...,216
173,2025-09-11-23-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...,203
355,2025-09-11-12-11-58 +0000,latimes,Charlie Kirk shooting reignites campus free-sp...,https://www.latimes.com/california/story/2025-...,183
263,2025-09-11-18-02-59 +0000,nyt,Trump Says He Will Award Charlie Kirk the Pres...,https://www.nytimes.com/2025/09/11/us/charlie-...,183
184,2025-09-11-22-28-28 +0000,nypost,Trump says he had ‘long talk’ with Charlie Kir...,https://nypost.com/2025/09/11/us-news/trump-sp...,177


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
54,216,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...
173,85,2025-09-11-23-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...
94,58,2025-09-12-06-29-26 +0000,wapo,Officials release new video of person fleeing ...,https://www.washingtonpost.com/nation/2025/09/...
223,49,2025-09-11-20-18-21 +0000,bbc,Bolsonaro's coup trial gripped Brazil - and hi...,https://www.bbc.com/news/articles/cj3yekj2xygo...
71,46,2025-09-12-08-38-23 +0000,nypost,Prince Harry makes surprise visit to Kyiv as h...,https://nypost.com/2025/09/12/world-news/princ...
141,43,2025-09-12-01-10-28 +0000,bbc,Man accused of Trump assassination attempt cut...,https://www.bbc.com/news/articles/c8d793z4mzgo...
74,40,2025-09-12-08-21-06 +0000,nypost,"Josh Kraft, son of billionaire Patriots owner,...",https://nypost.com/2025/09/12/us-news/robert-k...
225,40,2025-09-11-20-10-31 +0000,nypost,Russian drone incursion over Poland was ‘delib...,https://nypost.com/2025/09/11/world-news/russi...
127,37,2025-09-12-02-58-50 +0000,nyt,South Korean Leader Warns of U.S. Investments ...,https://www.nytimes.com/2025/09/11/world/asia/...
28,36,2025-09-12-10-18-00 +0000,wsj,Paramount Skydance Prepares Ellison-Backed Bid...,https://www.wsj.com/business/media/paramount-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
